In [ ]:
import numpy as np
import pandas as pd

import os

In [ ]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model

import matplotlib.pyplot as plt
import numpy as np
import cv2

In [ ]:
img_normal = cv2.imread("H:\\data\\Lung Disease Dataset\\train\\Normal\\07.jpeg")
img_tuber = cv2.imread("H:\\data\\Lung Disease Dataset\\train\\Tuberculosis\\test_0_14.jpeg")
img_covid = cv2.imread("H:\\data\\Lung Disease Dataset\\train\\Corona Virus Disease\\_8_7520889.jpeg")
img_bacterial = cv2.imread("H:\\data\\Lung Disease Dataset\\train\\Bacterial Pneumonia\\7.jpeg")
img_viral = cv2.imread("H:\\data\\Lung Disease Dataset\\train\\Viral Pneumonia\\03.jpeg")

In [ ]:
plt.imshow(img_normal)

In [ ]:
heatmap_normal = cv2.applyColorMap(img_normal, cv2.COLORMAP_JET)
heatmap_viral = cv2.applyColorMap(img_viral, cv2.COLORMAP_JET)
heatmap_bacterial = cv2.applyColorMap(img_bacterial, cv2.COLORMAP_JET)
heatmap_tuber = cv2.applyColorMap(img_tuber, cv2.COLORMAP_JET)
heatmap_covid = cv2.applyColorMap(img_covid, cv2.COLORMAP_JET)

In [ ]:
plt.imshow(heatmap_normal)

In [ ]:
plt.imshow(heatmap_viral)

In [ ]:
plt.imshow(heatmap_bacterial)

In [ ]:
plt.imshow(heatmap_tuber)

In [ ]:
plt.imshow(heatmap_covid)

In [ ]:
traingen = ImageDataGenerator(rescale= 1./255,
                             width_shift_range=0.2 , 
                             height_shift_range=0.2 ,
                             zoom_range=0.2)
valgen = ImageDataGenerator(rescale= 1./255)
testgen = ImageDataGenerator(rescale= 1./255)

In [ ]:
train_it = traingen.flow_from_directory("H:\\data\\Lung Disease Dataset\\train", target_size = (224, 224))
val_it = traingen.flow_from_directory("H:\\data\\Lung Disease Dataset\\val", target_size = (224, 224))
test_it = traingen.flow_from_directory("H:\\data\\Lung Disease Dataset\\test", target_size = (224, 224))


In [ ]:
plt.figure()
plt.imshow(next(train_it)[0][0])
plt.show()

In [ ]:
base_model_201 = tf.keras.applications.DenseNet201(input_shape = (224, 224, 3),
    include_top=False,
    weights="imagenet"
)

for layer in base_model_201.layers:
  layer.trainable = False

x = layers.Flatten()(base_model_201.output)  # base_model_201.output
x = layers.Dropout(0.5)(x) # 
x = layers.Dense(512, activation= 'relu')(x)
x = layers.Dense(5, activation = 'softmax')(x)

model2 = tf.keras.models.Model(base_model_201.input, x)  # keras.models not keras.model

model2.compile('adam', loss = 'categorical_crossentropy',metrics = ['acc'])

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

Checkpoint = tf.keras.callbacks.ModelCheckpoint("densenet201.hdf5", monitor="val_acc", save_best_only= True, mode="max")
reduce_lr = ReduceLROnPlateau(monitor= "val_acc", factor= 0.5, patience= 4, verbose= 3,cooldown=2,  mode= "max", min_lr= 0.0000001)

model2.fit(train_it, validation_data= val_it, epochs=1, callbacks= [reduce_lr, Checkpoint], steps_per_epoch=60, validation_steps=10)

In [ ]:
model_dense = keras.models.load_model('densenet201.hdf5')
model_dense.evaluate(test_it, steps= 1)

In [ ]:
pred = model_dense.predict(next(train_it)[0])
plt.imshow(pred)

In [ ]:
print(help(model_dense))

In [ ]:
predictions = []
for pred in model_dense.predict(next(test_it)[0]):
    p = []
    for i in pred:
        p.append(float(i))
    predictions.append(p)

In [ ]:
predictions